In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'evaluation'))

In [3]:
from scipy.io import arff
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import glob
from classification import compute_f1_score

In [4]:
def json_converter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()

# Correct F1 score in Json For Synthetic Data

In [7]:
data_set = 'mean'
algorithm = 'no_change'
results_path = os.path.join(os.path.dirname(os.getcwd()), 'results', 'test', algorithm, data_set)
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data', data_set)
list_of_results = glob.glob(os.path.join(results_path, "*.json"))
for result in list_of_results:
    with open(result) as file:
        results_dict = json.load(file)
    ts_name = results_dict['data_name']
    detected = np.array(results_dict['cp'])
    labels = pd.read_csv(os.path.join(data_path, "{}_labels.csv".format(ts_name)), header=None)
    corrected_f1 = compute_f1_score(labels.values[:-1, :], detected, 10)
    print(ts_name + " old: " + str(results_dict['score']['value']), " new: " + str(corrected_f1))
    results_dict['score']['value'] = corrected_f1
    with open(result, 'w') as file:
        json.dump(results_dict, file)

mean10 old: 0.2857142857142857  new: 0.33333333333333337
mean4 old: 0.2857142857142857  new: 0.33333333333333337
mean11 old: 0.2857142857142857  new: 0.33333333333333337
mean5 old: 0.2857142857142857  new: 0.33333333333333337
mean3 old: 0.2857142857142857  new: 0.33333333333333337
mean17 old: 0.2857142857142857  new: 0.33333333333333337
mean2 old: 0.2857142857142857  new: 0.33333333333333337
mean16 old: 0.2857142857142857  new: 0.33333333333333337
mean18 old: 0.2857142857142857  new: 0.33333333333333337
mean19 old: 0.2857142857142857  new: 0.33333333333333337
mean12 old: 0.2857142857142857  new: 0.33333333333333337
mean6 old: 0.2857142857142857  new: 0.33333333333333337
mean13 old: 0.2857142857142857  new: 0.33333333333333337
mean7 old: 0.2857142857142857  new: 0.33333333333333337
mean1 old: 0.2857142857142857  new: 0.33333333333333337
mean15 old: 0.2857142857142857  new: 0.33333333333333337
mean0 old: 0.2857142857142857  new: 0.33333333333333337
mean14 old: 0.2857142857142857  new: 0.

# Write table from jsons (REWRITES THE TABLE)

In [14]:
#Write results for sc
data_set = 'hasc'
algorithm = 'mssa'
results_path = os.path.join(os.path.dirname(os.getcwd()), 'results', 'test', algorithm, data_set)
list_of_results = glob.glob(os.path.join(results_path, "*.json"))
table = []
for result in list_of_results:
    with open(result, 'r') as file:
        results_dict = json.load(file)
    ts_name = results_dict['data_name']
    status = results_dict['status']
    score = results_dict['score']
    if score: 
        score = score['value']
    else: 
        continue 
    row = [algorithm, data_set, ts_name, status, score]
    table.append(row)
table = pd.DataFrame(table)
table.to_csv('{}/results_table.csv'.format(results_path), index=False, header=False)
